# 1-scraping

In [17]:
# Manejo de datos 
import pandas as pd # manejo de datos y dataframes
import numpy as np # manejo de arrays y operaciones matematicas 


# Librerias para realizar web scraping con selenium
from selenium import webdriver # webdriver permite manejar un navegador 
from webdriver_manager.chrome import ChromeDriverManager # permite instalar y mantener actualizado el driver de chrome
from selenium.webdriver.common.keys import Keys # permite simular teclas del teclado 
from selenium.webdriver.chrome.options import Options # permite configurar el driver de chrome como modo incognito o maximizar la ventana
from time import sleep # esperas entre ejecuciones de codigo


import warnings # permite ignorar los warnings de python 
warnings.filterwarnings('ignore')

In [18]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime

In [19]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') # empezar maximizado
opciones.add_argument('user.data-dir=selenium') # guarda las cookies
opciones.add_argument('--incognito') # incognito window

# selenium

In [20]:
urls = []
flag = 0
flag2 = 0

driver = webdriver.Chrome()

driver.get("https://www.dia.es/") 
sleep(3)

# aceptamos las cookies 
driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
sleep(2)

# clicka productos
driver.find_element("css selector", '#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > div > button').click()
sleep(2)

# para cada una de las categorías principales
# poner un max excesivo
for category in range(1,31):

    while 1:

        # intenta clickar
        try:
            
            driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/a').click()
            flag2 = 0

            for subcategory in range (1,31):

                while 1:

                    try:
                        
                        urls.append(driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div/div/div[2]/ul/li[{category}]/ul/div[{subcategory}]/a').get_attribute("href"))
                        sleep(0.1)
                        break

                    except:

                        flag2 = 1
                        break
                    
                if flag2:
                    break
            
            sleep(1)
            break
        
        # break si se acaban
        except:

            flag = 1
            break

    if flag:

        break

driver.quit()

In [ ]:
df = pd.DataFrame(urls, columns=['url'])

df.to_csv("../data/urls-dia.csv", index=False, sep= ",")

# sopa bonita

In [ ]:
# ['url', 'supermarket', 'category', 'name', 'description', 'price', 'reference_price', 'reference_unit', 'insert_date', 'product_id']

resultados_dia = {'url': 'https://www.dia.es/categoria/subcategoria/p/id',
                "supermarket": 'dia-es',
                "category":[],
                'subcategory':[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": datetime.today().strftime('%Y-%m-%d'),
                "product_id": []}

# solo saca 10/20

In [ ]:
url = 'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001'
res = requests.get(url)
sleep(1)

print(res.status_code)

sopa = BeautifulSoup(res.content, 'html.parser')

#productos = sopa.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
productos = sopa.find_all("p", {"class": "search-product-card__product-name"})

print(f'Cantidad de productos extraídos: {len(productos)}')

print(productos[0])

print(productos[0].text)

productos


# scroll por pixel saca 20/20

In [ ]:
# Create your driver
driver = webdriver.Chrome()

# Get a page
driver.get("https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001") 
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos = soup.find_all("p", {"class": "search-product-card__product-name"})
precio = soup.find_all("p", {"class": "search-product-card__active-price"})
kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})

driver.quit()

print(f'Productos extraídos: {len(productos)} de 20')

print(productos[-1].text)
print(precio[-1].text)
print(kilo[-1].text)

# crea una lista de listas, cada lista es una página de productos

In [ ]:
productos = []
precio = []
kilo = []

# Create your driver
driver = webdriver.Chrome()

# Get a page
driver.get("https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001")
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos = soup.find_all("p", {"class": "search-product-card__product-name"})
precio = soup.find_all("p", {"class": "search-product-card__active-price"})
kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})

driver.quit()

print(f'Productos extraídos: {len(productos)} de 20')

print(productos)
print(precio)
print(kilo)

# bucle para meter todas las páginas de un producto

In [ ]:
productos = []
precio = []
kilo = []
flag3 = 0

# Create your driver
driver = webdriver.Chrome()

param = 'c'
url = f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001'

# Get a page
driver.get(url)
sleep(2)

Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# Feed the source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

for i in range(2,10):

    param = f'pag-{i}'
    url = f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/c/L2001'

    driver.get(url)
    sleep(2)

    Y = 1200
    driver.execute_script(f"window.scrollTo(0, {Y})") 
    sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    if soup.find_all("div", {"class": "plp-error-page__card"}):

        break

    productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
    precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
    kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

driver.quit()

print(len(productos))
print(precio)
print(kilo)

# bucle para meter varias urls

In [ ]:
productos = []
precio = []
kilo = []
flag3 = 0

# creación del driver
driver = webdriver.Chrome()

# c = página 1
param = 'c'
url = [f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001', 
       f'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/{param}/L2206']

# get
driver.get(u)
sleep(2)

# scroll down
Y = 1200
driver.execute_script(f"window.scrollTo(0, {Y})") 
sleep(2)

# parseo html para sopa
soup = BeautifulSoup(driver.page_source, 'html.parser')

# apendea los resultados de la página 1
productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

# itero i para modificar url
for i in range(2,10):
    
    param = f'pag-{i}/c'

    # para cada url de la lista:
    for u in url:

        driver.get(u)
        sleep(2)

        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if soup.find_all("div", {"class": "plp-error-page__card"}):

            break

        productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

driver.quit()

print(len(productos))
print(precio)
print(kilo)

In [ ]:
productos = []
precio = []
kilo = []

# creación del driver
driver = webdriver.Chrome()

for i in range(2,11):

    if i == 1:

        param = 'c'

    else:

        param = f'pag-{i}/c'

    url = [f'https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/{param}/L2001', f'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/{param}/L2206']

    for u in url:

        # get
        driver.get(u)
        sleep(2)

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        productos.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

In [27]:
categoria = []
subcategoria = []
producto = []
precio = []
kilo = []
id = []

# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001', 'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206']

for u in url:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        id.append(soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"}))
        categoria.append(soup.find("span", {"class": "plp-breadcrumb__first-level-category"}))
        subcategoria.append(soup.find("span", {"class": "plp-breadcrumb__second-level-category"}))
        producto.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        if botones:

                for bottom in range(2, int(botones[-1].text)+1):
                        
                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                        sleep(2)

                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        id.append(soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"}))
                        categoria.append(soup.find("span", {"class": "plp-breadcrumb__first-level-category"}))
                        subcategoria.append(soup.find("span", {"class": "plp-breadcrumb__second-level-category"}))
                        producto.append(soup.find_all("p", {"class": "search-product-card__product-name"}))
                        precio.append(soup.find_all("p", {"class": "search-product-card__active-price"}))
                        kilo.append(soup.find_all("p", {"class": "search-product-card__price-per-unit"}))

        botones = []

driver.quit()

print(len(producto))
print(categoria)
print(subcategoria)
print(producto)
print(precio)
print(kilo)
print(id)




4
[<span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>, <span class="plp-breadcrumb__first-level-category">Charcutería y quesos</span>]
[<span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Jamón cocido, lacón, fiambres y mortadela</span>, <span class="plp-breadcrumb__second-level-category">Salchichas y elaborados</span>]
[[<p class="search-product-card__product-name">Pechuga de pavo NUESTRA ALACENA pack 2 unidades SOBRE 400 GR</p>, <p class="search-product-card__product-name">Jamón cocido extra NUESTRA ALACENA pack 2 unidades SOBRE 450 GR</p>, <p class="search-product-card__product-name">Bacon en tir

In [ ]:
# ['url', 'supermarket', 'category', 'name', 'description', 'price', 'reference_price', 'reference_unit', 'insert_date', 'product_id']

resultados_dia = {'url': 'https://www.dia.es/categoria/subcategoria/p/id',
                "supermarket": 'dia-es',
                "category":[],
                'subcategory':[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": datetime.today().strftime('%Y-%m-%d'),
                "product_id": []}

resultados_dia;


In [ ]:
# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001', 
       'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206']

for u in url:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        for p in prod:
                resultados_dia['category'].append(cat.text)
                resultados_dia['subcategory'].append(sub.text)
                resultados_dia['name'].append(p.text)
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                resultados_dia['reference_price'].append(k.text)
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):
                        
                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                        sleep(2)

                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['category'].append(cat.text)
                                resultados_dia['subcategory'].append(sub.text)
                                resultados_dia['name'].append(p.text)
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                resultados_dia['reference_price'].append(k.text)
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        botones = []

driver.quit()

resultados_dia;

In [ ]:
resultados_dia = {'url': 'https://www.dia.es/categoria/subcategoria/p/id',
                "supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": [],
                "product_id": []}

# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001', 
       'https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206']

for u in url:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):
                        
                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                        sleep(2)

                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        botones = []

driver.quit()

resultados_dia;

In [26]:
resultados_dia = {'url': 'https://www.dia.es/categoria/subcategoria/p/id',
                "supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "description": [], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": [],
                "product_id": []}

resultados_dia = {"supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": []}

# creación del driver
driver = webdriver.Chrome()

for u in urls:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):
                        
                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/div/div/div[{bottom}]/a').click()
                                
                        
                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        botones = []

driver.quit()

resultados_dia

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0092A813+48355]
	(No symbol) [0x008BC4B1]
	(No symbol) [0x007C5358]
	(No symbol) [0x007AD293]
	(No symbol) [0x0080E37B]
	(No symbol) [0x0081C473]
	(No symbol) [0x0080A536]
	(No symbol) [0x007E82DC]
	(No symbol) [0x007E93DD]
	GetHandleVerifier [0x00B8AABD+2539405]
	GetHandleVerifier [0x00BCA78F+2800735]
	GetHandleVerifier [0x00BC456C+2775612]
	GetHandleVerifier [0x009B51E0+616112]
	(No symbol) [0x008C5F8C]
	(No symbol) [0x008C2328]
	(No symbol) [0x008C240B]
	(No symbol) [0x008B4FF7]
	BaseThreadInitThunk [0x76C07D59+25]
	RtlInitializeExceptionChain [0x77E1B74B+107]
	RtlClearBits [0x77E1B6CF+191]


In [28]:
resultados_dia

{'supermarket': ['dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es'],
 'category': ['charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',
  'charcuteria_y_quesos',


In [25]:
df2 = pd.DataFrame(resultados_dia)

df2

ValueError: All arrays must be of the same length

In [35]:
resultados_dia = {"supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": []}

# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206',
       'https://www.dia.es/conservas-caldos-y-cremas/conservas-vegetales/c/L2092']

for u in url:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                pass

                
                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        botones = []

driver.quit()

resultados_dia

{'supermarket': ['dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
 

In [39]:
resultados_dia = {"supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": []}

# creación del driver
driver = webdriver.Chrome()

url = ['https://www.dia.es/charcuteria-y-quesos/salchichas-y-elaborados/c/L2206',
       'https://www.dia.es/conservas-caldos-y-cremas/conservas-vegetales/c/L2092']

for u in url:

        # get
        driver.get(u)
        sleep(2)

        # aceptamos las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parseo html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # apendea los resultados de la página 1
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                try:
                                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/a[2]').click()
                                        sleep(2)
                                except:
                                        pass

                                pass
                        

                
                        # scroll down
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        # parseo html para sopa
                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # apendea los resultados de la página 1
                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        botones = []

driver.quit()

resultados_dia

{'supermarket': ['dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
 

In [40]:
# dictio con los resultados del scrapeo
resultados_dia = {"supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": []}

# creación del driver de chrome
driver = webdriver.Chrome()

# para cada uno de los urls que han salido del primer Selenium
for u in urls:

        # get
        driver.get(u)
        sleep(2)

        # acepta las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parsea html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # listas resultados primera página
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        # apendea los resultados de las listas al dictio extrayendo texto de labels y retocando con métodos de strings
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        # intenta clickar el botón
                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                # para cuando hay botones flecha (+5 páginas)
                                try:
                                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/a[2]').click()
                                        sleep(2)
                                except:
                                        pass

                                pass
                        
                        # SE REPITE EL PROCESO ANTERIOR PARA EL RESTO DE PÁGINAS:
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        # resetea nº botones
        botones = []

# cierra driver
driver.quit()

# show
resultados_dia

{'supermarket': ['dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
  'dia-es',
 

In [42]:
df2 = pd.DataFrame(resultados_dia)

df2.to_csv("../scrapeo.csv", index=False, sep= ",")

In [43]:
df2

,supermarket,category,subcategory,name,price,reference_price,reference_unit,insert_date
0,dia-es,charcuteria_y_quesos,jamon_cocido_lacon_fiambres_y_mortadela,Pechuga de pavo NUESTRA ALACENA pack 2 unidade...,"3,55","8,88",kilo,2023-07-07
1,dia-es,charcuteria_y_quesos,jamon_cocido_lacon_fiambres_y_mortadela,Jamón cocido extra NUESTRA ALACENA pack 2 unid...,"3,59","7,98",kilo,2023-07-07
2,dia-es,charcuteria_y_quesos,jamon_cocido_lacon_fiambres_y_mortadela,Bacon en tiras NUESTRA ALACENA pack 2 unidades...,"1,79","8,95",kilo,2023-07-07
3,dia-es,charcuteria_y_quesos,jamon_cocido_lacon_fiambres_y_mortadela,Bacon NUESTRA ALACENA SOBRE 200 GR,"1,99","9,95",kilo,2023-07-07
4,dia-es,charcuteria_y_quesos,jamon_cocido_lacon_fiambres_y_mortadela,Jamón cocido extra 97% carne NUESTRA ALACENA ...,"2,05","13,67",kilo,2023-07-07
...,...,...,...,...,...,...,...,...
6185,dia-es,mascotas,gatos,Finas láminas surtidas en salsa para gatos GOU...,"3,79","11,15",kilo,2023-07-07
6186,dia-es,mascotas,gatos,Alimento para gatos esterilizados rico en buey...,"5,69","7,11",kilo,2023-07-07
6187,dia-es,mascotas,otros_animales,Barritas para roedores de verdura y fruta BIOZ...,"1,79","15,98",kilo,2023-07-07
6188,dia-es,mascotas,otros_animales,Barritas con fruta para canarios BIAZOO BOLS...,"1,49","24,83",kilo,2023-07-07


In [ ]:
# dictio con los resultados del scrapeo
resultados_dia = {"supermarket": [],
                "category":[],
                'subcategory':[],
                "name":[], 
                "price": [], 
                "reference_price": [],
                "reference_unit": [],
                "insert_date": []}

# creación del driver de chrome
driver = webdriver.Chrome()

# para cada uno de los urls que han salido del primer Selenium
for u in urls:

        # get
        driver.get(u)
        sleep(2)

        # acepta las cookies
        try:
                driver.find_element("css selector", '#onetrust-accept-btn-handler').click()
                sleep(1)

        except:
                pass

        # scroll down
        Y = 1200
        driver.execute_script(f"window.scrollTo(0, {Y})") 
        sleep(2)

        # parsea html para sopa
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # listas resultados primera página
        cat = soup.find("span", {"class": "plp-breadcrumb__first-level-category"})
        sub = soup.find("span", {"class": "plp-breadcrumb__second-level-category"})
        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
        #id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
        
        # apendea los resultados de las listas al dictio extrayendo texto de labels y retocando con métodos de strings
        for p in prod:
                resultados_dia['supermarket'].append('dia-es')
                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                resultados_dia['name'].append(p.text)
                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
        
        for o in precio:
                resultados_dia['price'].append(o.text.rstrip('\xa0€'))
        
        for k in kilo:
                lst = k.text.split('\xa0€/')
                resultados_dia['reference_price'].append(lst[0].lstrip(' ('))
                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
        
        #for i in id:
                #resultados_dia['product_id'].append(i.text)
        
        # si hay botones, los encuentra
        if bool(soup.find_all("a", {"class": "pagination-button__page--links"})):
                botones = soup.find_all("a", {"class": "pagination-button__page--links"})

        else:
                pass
        
        # si hay botones (más páginas):
        if botones:

                # para cada botón siguiente (a partir del 2) hasta que se acaben:
                for bottom in range(2, int(botones[-1].text)+1):

                        # intenta clickar el botón
                        try:
                                driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div/div/div/div[{bottom}]/a').click()
                                sleep(2)
                        
                        except:
                                # para cuando hay botones flecha (+5 páginas)
                                try:
                                        driver.find_element("xpath", f'//*[@id="app"]/div/div/div/div[2]/div[2]/div[3]/div[2]/div[1]/a[2]').click()
                                        sleep(2)
                                except:
                                        pass

                                pass
                        
                        # SE REPITE EL PROCESO ANTERIOR PARA EL RESTO DE PÁGINAS:
                        driver.execute_script(f"window.scrollTo(0, {Y})") 
                        sleep(2)

                        soup = BeautifulSoup(driver.page_source, 'html.parser')

                        prod = soup.find_all("p", {"class": "search-product-card__product-name"})
                        precio = soup.find_all("p", {"class": "search-product-card__active-price"})
                        kilo = soup.find_all("p", {"class": "search-product-card__price-per-unit"})
                        id = soup.find_all("div", {"class": "search-product-card search-product-card__dia-border product-card-list__item"})
                        
                        for p in prod:
                                resultados_dia['supermarket'].append('dia-es')
                                resultados_dia['category'].append(cat.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['subcategory'].append(sub.text.lower().replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace(',', '').replace(" ", "_"))
                                resultados_dia['name'].append(p.text)
                                resultados_dia['insert_date'].append(datetime.today().strftime('%Y-%m-%d'))
                        
                        for o in precio:
                                resultados_dia['price'].append(o.text.rstrip('\xa0€').replace(',', '.'))
                        
                        for k in kilo:
                                lst = k.text.split('\xa0€/')
                                resultados_dia['reference_price'].append(lst[0].lstrip(' (')).replace(',', '.')
                                resultados_dia['reference_unit'].append(lst[1].rstrip(') ').lower())
                        
                        #for i in id:
                                #resultados_dia['product_id'].append(i.text)

        # resetea nº botones
        botones = []

# cierra driver
driver.quit()

# show
resultados_dia